In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from pandas_profiling import ProfileReport
import datetime as dt
from lifetimes import BetaGeoFitter

In [84]:
#pip install Lifetimes==0.2.2.2

     |████████████████████████████████| 500 kB 1.9 MB/s eta 0:00:01
  Created wheel for Lifetimes: filename=Lifetimes-0.2.2.2-py3-none-any.whl size=521868 sha256=c76f94353a08a2f6f5ceddb5474e4a2e68fe016345bdecbc253eb5e0f81ef099
  Stored in directory: /Users/nareabgaryan/Library/Caches/pip/wheels/27/3f/7f/ddb390ca9a8db19640cded9631c1f90935dc521826c764b7ab
Successfully built Lifetimes
Note: you may need to restart the kernel to use updated packages.


# Data Preperation

In [44]:
data = pd.read_excel("Dataset.xlsx")

In [45]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [46]:
filtered_data = data.drop_duplicates()
filtered_data = filtered_data[(filtered_data['Quantity']>0)]
filtered_data = filtered_data [['Customer ID','Description','InvoiceDate','Invoice','Quantity','Price', 'Country']]
filtered_data['TotalPurchase'] = filtered_data['Quantity'] * filtered_data['Price']

In [52]:
filtered_data_group = filtered_data.groupby(['Customer ID','Country']).agg({'InvoiceDate': lambda date: (date.max() - date.min()).days,
                                        'Invoice': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'TotalPurchase': lambda price: price.sum()    })

In [53]:
filtered_data_group.columns=['num_days','num_transactions','num_units','spent_money']

In [26]:
filtered_data_group['avg_order_value'] = filtered_data_group['spent_money']/filtered_data_group['num_transactions']

In [27]:
purchase_frequency = sum(filtered_data_group['num_transactions'])/filtered_data_group.shape[0]

In [28]:
repeat_rate = round(filtered_data_group[filtered_data_group.num_transactions > 1].shape[0]/filtered_data_group.shape[0],2)

In [29]:
churn_rate = round(1-repeat_rate,2)

In [30]:
filtered_data_group.reset_index()

,Customer ID,Country,num_days,num_transactions,num_units,spent_money,avg_order_value
0,12346.0,United Kingdom,196,33,70,372.86,11.298788
1,12347.0,Iceland,37,71,828,1323.32,18.638310
2,12348.0,Finland,0,20,373,222.16,11.108000
3,12349.0,Italy,181,102,993,2671.14,26.187647
4,12351.0,Unspecified,0,21,261,300.93,14.330000
...,...,...,...,...,...,...,...
4314,18283.0,United Kingdom,275,217,322,619.37,2.854240
4315,18284.0,United Kingdom,0,28,494,461.68,16.488571
4316,18285.0,United Kingdom,0,12,145,427.00,35.583333
4317,18286.0,United Kingdom,247,67,608,1296.43,19.349701


In [31]:
filtered_data_group['profit_margin'] = filtered_data_group['spent_money']*0.05

In [32]:
filtered_data_group['CLV'] = (filtered_data_group['avg_order_value']*purchase_frequency)/churn_rate

In [33]:
filtered_data_group.reset_index(inplace = True)

In [34]:
filtered_data_group['spent_money', 'avg_order_value','profit_margin'] = filtered_data_group.spent_money.apply(lambda x : "{:,}".format(x))

In [69]:
data.dropna(inplace=True)
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])
data["InvoiceDate"].max()
today_date = dt.datetime(2011, 12, 11)
data["TotalPrice"] = data["Price"] * data["Quantity"]

In [70]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [73]:
rfm = data.groupby("Customer ID").agg({"InvoiceDate": lambda InvıiceDate: (today_date- InvıiceDate.max()).days,
                                    "Invoice": lambda Invoice: Invoice.nunique(),
                                    "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["recency","frequency","monetary"]
rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4282.0,453.560953,95.734145,3.660000e+02,382.0000,415.00,495.0000,739.00
frequency,4282.0,5.455395,10.148765,1.000000e+00,1.0000,3.00,6.0000,270.00
monetary,4282.0,1971.709950,8596.562848,1.776357e-15,303.6925,686.65,1683.9625,341776.73


In [101]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

# Model

In [91]:
#BG/NBD Model 
cltv_df = data.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [97]:
bgf = BetaGeoFitter()

In [99]:
#Gamma model
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 3080 subjects, p: 0.60, q: 5.54, v: 2379.86>